In [ ]:
import os
import subprocess

CONDACI_URL_START = 'https://raw.githubusercontent.com/menpo/condaci/v'

PROJECTS = ['menpo', 
            'menpofit', 
            'menpodetect', 
            'menpo3d', 
            'menpobench',
            'landmarkerio-server',
            'cyassimp',
            'cyrasterize',
            'cyvlfeat',
            'cyffld2',
            'cypico',
            'conda-dlib', 
            'conda-boost', 
            'conda-joblib', 
            'conda-enum', 
            'conda-flann',
            'conda-cherrypy',
            'conda-pathlib',
            'conda-eigen',
            'vrml97']

def load_file(fpath):
    with open(fpath, 'rt') as f:
        text = f.read()
    return text


def save_file(fpath, string):
    with open(fpath, 'wt') as f:
        f.write(string)
        
        
def replace_version_in_str(string, v):
    before, after = string.split(CONDACI_URL_START)
    before += CONDACI_URL_START

    old_v, after = after.split('/', 1)
    after = '/' + after
    return before + v + after, old_v


def replace_version(fpath, v):
    text = load_file(fpath)
    fixed, old_v = replace_version_in_str(text, v)
    print('{} -> {}  {}'.format(old_v, v, fpath))
    save_file(fpath, fixed)
    return old_v
    
    
def replace_version_in_project(repo_dir, v):
    travis = os.path.join(repo_dir, '.travis.yml')
    old_v_t = replace_version(travis, v)
    appveyor = os.path.join(repo_dir, 'appveyor.yml')
    old_v_a = replace_version(appveyor, v)
    if old_v_a == old_v_t == v:
        return False
    else:
        return True
    

def bump_repo(repo_dir, v):
    os.chdir(repo_dir)
    if replace_version_in_project(repo_dir, v):
        print(subprocess.check_output(['git', 'commit', '-am', 'bump condaci to {}'.format(v)]))
        print('bumped {}'.format(repo_dir))
    else:
        print('{} is already up to date'.format(repo_dir))
    subprocess.check_output(['git', 'push', 'origin', 'master'])


def repo_url(project_name):
    return 'git@github.com:menpo/{}'.format(project_name)
    
    
def clone_repo(project_name):
    repo_url = 'git@github.com:menpo/{}'.format(project_name)
    print('cloning {}'.format(repo_url))
    subprocess.check_output(['git', 'clone', repo_url])

In [ ]:
v = '0.3.0'
working_dir = '/Users/jab08/bump_condaci'

In [ ]:
if not os.path.isdir(working_dir):
    os.mkdir(working_dir)
os.chdir(working_dir)

In [ ]:
from subprocess import CalledProcessError

for project in PROJECTS:
    os.chdir(working_dir)
    try:
        clone_repo(project)
    except CalledProcessError:
        print('repo already cloned')
        pass
    repo_dir = os.path.join(working_dir, project)
    print('attempting to bump {}'.format(project))
    bump_repo(repo_dir, v)